In [1]:
!wget -O trainingset.zip https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1

--2025-01-03 07:51:27--  https://zenodo.org/records/10450641/files/trainingset-ideology-power.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.45.92, 188.185.43.25, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.45.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 813899321 (776M) [application/octet-stream]
Saving to: ‘trainingset.zip’

trainingset.zip     100%[===================>] 776.19M  13.1MB/s    in 63s     

2025-01-03 07:52:31 (12.3 MB/s) - ‘trainingset.zip’ saved [813899321/813899321]



In [2]:
!wget -O testset.zip https://zenodo.org/records/11061650/files/ideology-power-st-testset.zip?download=1

--2025-01-03 07:52:31--  https://zenodo.org/records/11061650/files/ideology-power-st-testset.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 268140953 (256M) [application/octet-stream]
Saving to: ‘testset.zip’

testset.zip         100%[===================>] 255.72M  12.6MB/s    in 21s     

2025-01-03 07:52:52 (12.1 MB/s) - ‘testset.zip’ saved [268140953/268140953]



In [3]:
!unzip trainingset.zip -d trainingset

Archive:  trainingset.zip
   creating: trainingset/orientation/
  inflating: trainingset/orientation/orientation-at-train.tsv  
  inflating: trainingset/orientation/orientation-ba-train.tsv  
  inflating: trainingset/orientation/orientation-be-train.tsv  
  inflating: trainingset/orientation/orientation-bg-train.tsv  
  inflating: trainingset/orientation/orientation-cz-train.tsv  
  inflating: trainingset/orientation/orientation-dk-train.tsv  
  inflating: trainingset/orientation/orientation-ee-train.tsv  
  inflating: trainingset/orientation/orientation-es-ct-train.tsv  
  inflating: trainingset/orientation/orientation-es-ga-train.tsv  
  inflating: trainingset/orientation/orientation-es-train.tsv  
  inflating: trainingset/orientation/orientation-fi-train.tsv  
  inflating: trainingset/orientation/orientation-fr-train.tsv  
  inflating: trainingset/orientation/orientation-gb-train.tsv  
  inflating: trainingset/orientation/orientation-gr-train.tsv  
  inflating: trainingset/orientati

In [4]:
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv("./trainingset/power/power-tr-train.tsv", delimiter = "\t")
train_df, test_df = train_test_split(
    df, test_size=0.1, stratify=df['label'], random_state=42
)
print(train_df)


            id                           speaker sex  \
7838   tr25984  9f2161636a7c1bbd385e7893bc0d9c85   M   
11440  tr29586  e20ee998a6c5cb6a6b95034dfa2808ba   M   
10808  tr28954  2ead8c44e83722110aeafcda867a8ba3   M   
12133  tr30279  9ea3207b08f5e93042b3b8ef4fa7b7b3   M   
14530  tr32676  2e5b14b14f72f3d24f86401121a0c37a   F   
...        ...                               ...  ..   
15943  tr34089  498dcde0a7f7faee74e604562258ddb3   F   
6269   tr24415  043d8ce9b522d8896d08711df295a5e7   M   
7115   tr25261  d69dee113e92cbf04647c0565ccc4da3   F   
14666  tr32812  1324e52f6dc5a4677e80b2246090910f   M   
9110   tr27256  a71f9077d9a1f752c5dacedc685f635d   M   

                                                    text  \
7838   Değerli arkadaşlar, bakın, bugün enerjide önem...   
11440  Bu bütçeye “evet” demek Kürt emekçilerine “Dil...   
10808  Sayın Başkanım, vekil arkadaşımızın ifade etmi...   
12133  Sayın Başkan, bir cevap verebilir miyim? <p> “...   
14530  Sayın Başkan, değerl

In [5]:
from transformers import XLMRobertaForSequenceClassification, AutoTokenizer

mymodel_bert= "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(mymodel_bert)
model = XLMRobertaForSequenceClassification.from_pretrained(mymodel_bert, num_labels=2)
def tokenize_dataset(examples):
    return bert_tokenizer(examples["text"].tolist(), padding="max_length", truncation=True)

train_encodings = tokenize_dataset(train_df)
test_encodings = tokenize_dataset(test_df)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch.nn as nn
classifier = nn.Linear(768, 2)


In [ ]:
from transformers import AdamW
criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import torch
train_inputs = [bert_tokenizer(text, padding="max_length", truncation=True, return_tensors="pt") for text in train_df['text_en']]
test_inputs = [bert_tokenizer(text, padding="max_length", truncation=True, return_tensors="pt") for text in test_df['text_en']]

train_dataset = torch.utils.data.TensorDataset(
    torch.cat([x['input_ids'] for x in train_inputs]),
    torch.cat([x['attention_mask'] for x in train_inputs]),
    torch.tensor(list(train_df['label']))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.cat([x['input_ids'] for x in test_inputs]),
    torch.cat([x['attention_mask'] for x in test_inputs]),
    torch.tensor(list(test_df['label']))
)

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
from torch.optim import AdamW,SGD
from transformers import get_scheduler
num_epochs=3
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=1, num_training_steps=num_training_steps
)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_loader:
        inputs, attention_mask, labels = batch
        batch = {}
        batch["input_ids"] = inputs.to(device)
        batch["attention_mask"] = attention_mask.to(device)
        batch["labels"]= labels.to(device)


        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/1467 [00:00<?, ?it/s]

In [ ]:
!pip install evaluate
import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in test_loader:
    inputs, attention_mask, labels = batch
    batch = {}
    batch["input_ids"] = inputs.to(device)
    batch["attention_mask"] = attention_mask.to(device)
    batch["labels"]= labels.to(device)

    b = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**b)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


{'accuracy': 0.8924669350201265}

In [6]:
import pandas as pd
from transformers import pipeline
from sklearn.model_selection import train_test_split

classifier = pipeline(
    "zero-shot-classification",
    model="bigscience/bloom-560m",
    device=0
)

labels = ["Label 0", "Label 1"]

test_texts = test_df["text_en"].tolist()[:100]  #for gpu constraints i leave only 100 texts, but tried more and saw it works
results = classifier(test_texts, candidate_labels=labels)

for i, res in enumerate(results):
    print(f"Text: {test_texts[i]}")
    print(f"Prediction: {res['labels'][0]} with score {res['scores'][0]:.4f}")
    print(f"Prediction: {res['labels'][1]} with score {res['scores'][1]:.4f}")
    print("-" * 50)


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of BloomForSequenceClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Text: Thank you, Mr. President. <p> Of course, since it's the Justice Commission, the Minister of Justice is here, which is actually an arrangement that concerns the Department of Labor. For him, Mr. Mehmet Mush has just responded here, breaking the rules, instead of the government, as he is the Deputy Director of the Group, but he doesn't know anything. After all, I'm already telling Mr. Mush, and I want him to come and answer -- I'm not going to stick to my speech on matter, I prepared him -- because in his speech, we, the <PARTY>, have done a lot for the workers for the past five years. <p> I mean, do we live in a different country? <p> Dear friends, I would argue that the <PARTY> has not made or passed a single clause in favor of the worker in 15 years. After this conversation, Mehmet Mush will come up here and we'll pass this clause. He says, "You'll witness it here." I'll tell you: when they first arrived in 2002, they first changed the Law on Business Safety. Ten people and thos